In [ ]:
from pytube import YouTube
from innertube import InnerTube
from datetime import timedelta


def extract_transcript_params(next_data):
    engagement_panels = next_data["engagementPanels"]

    for engagement_panel in engagement_panels:
        engagement_panel_section = engagement_panel[
            "engagementPanelSectionListRenderer"
        ]

        if (
            engagement_panel_section.get("panelIdentifier")
            != "engagement-panel-searchable-transcript"
        ):
            continue

        return engagement_panel_section["content"]["continuationItemRenderer"][
            "continuationEndpoint"
        ]["getTranscriptEndpoint"]["params"]


def get_chapters(transcript):
    segments = transcript["actions"][0][
        "updateEngagementPanelAction"]["content"][
        "transcriptRenderer"]["content"][
        "transcriptSearchPanelRenderer"]["body"][
        "transcriptSegmentListRenderer"]["initialSegments"]
    
    chapters = []
    for segment in segments:
        if segment.get("transcriptSectionHeaderRenderer"):
            section = segment["transcriptSectionHeaderRenderer"]
            chapters.append(dict(
                    start=float(section["startMs"]) / 1000.,
                    end=float(section["endMs"]) / 1000.,
                    label=section["accessibility"]["accessibilityData"]["label"]
            ))
    return chapters 

yt = YouTube("https://www.youtube.com/watch?v=1Dx7LDwINLU&t=327s")
it = InnerTube(client_name="WEB")
data = it.next(yt.video_id)
params = extract_transcript_params(data)
transcript = it.get_transcript(params)
chapters = get_chapters(transcript)

for chapter in chapters:
    fancy_start_tm = timedelta(seconds=chapter["start"])
    print(f"{fancy_start_tm} {chapter['label']}")

0:00:00 Intro
0:00:46 Monomer Definition
0:01:04 Carbohydrates
0:02:45 Lipids
0:04:24 Proteins
0:05:37 Nucleic Acids
0:06:38 Biomolecule Structure
